In [2]:
import pandas as pd

In [55]:
delay_df = pd.read_excel("/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/deploy_ml/get_around_delay_analysis.xlsx")
pricing_df = pd.read_csv("/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/deploy_ml/get_around_pricing_project.csv")

In [56]:
pricing_df.drop('Unnamed: 0', axis=1, inplace=True)

In [57]:
(pricing_df["has_getaround_connect"] == True).sum()

2230

In [58]:
delay_df.dtypes

rental_id                                       int64
car_id                                          int64
checkin_type                                   object
state                                          object
delay_at_checkout_in_minutes                  float64
previous_ended_rental_id                      float64
time_delta_with_previous_rental_in_minutes    float64
dtype: object

In [72]:
pricing_df[(pricing_df['engine_power'] < 100) & (pricing_df['mileage'] < 12000)]


,mileage,engine_power,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day,model_key_encoded,car_type_encoded,fuel_type_encoded,paint_color_encoded
25,196092,85,True,False,False,False,False,False,False,42,19,0,3,2
58,265751,85,False,False,False,False,False,False,False,32,21,1,3,8
157,146310,85,False,False,False,False,False,False,True,87,21,2,0,1
163,158002,85,False,False,False,False,False,False,True,94,21,2,0,9
166,135119,85,False,True,False,False,False,False,True,104,21,2,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,29765,85,False,False,False,False,False,False,True,127,25,7,0,5
4826,28573,70,True,True,False,False,False,False,True,133,27,7,0,7
4829,38334,85,False,False,False,False,False,False,True,127,25,7,0,9
4830,48012,85,True,True,False,False,False,True,True,125,25,7,0,2


In [60]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the label encoder on the car model column
pricing_df['model_key_encoded'] = label_encoder.fit_transform(pricing_df['model_key'])
pricing_df['car_type_encoded'] = label_encoder.fit_transform(pricing_df['car_type'])
pricing_df['fuel_type_encoded'] = label_encoder.fit_transform(pricing_df['fuel'])
pricing_df['paint_color_encoded'] = label_encoder.fit_transform(pricing_df['paint_color'])

In [61]:
pricing_df.drop(['model_key', 'car_type', 'fuel', 'paint_color'], axis=1, inplace=True)

In [ ]:
#pricing_df.to_csv('pricing_df.csv', index=False)

In [62]:
correlation = pricing_df.corr()['rental_price_per_day'].abs().sort_values(ascending=False)
max_correlation_feature = correlation.index[1]  # Exclude 'rental_price_per_day' itself
max_correlation_value = correlation[1]

/var/folders/lt/pb3vc41d15l30zltrtdn76b00000gn/T/ipykernel_76291/313653561.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [65]:
correlation #most corelated features with price that I ll use for the dashboard

rental_price_per_day         1.000000
engine_power                 0.625645
mileage                      0.448912
automatic_car                0.419761
has_getaround_connect        0.318486
has_gps                      0.310889
private_parking_available    0.281358
car_type_encoded             0.262654
has_air_conditioning         0.245386
has_speed_regulator          0.227547
model_key_encoded            0.089356
fuel_type_encoded            0.055321
winter_tires                 0.018277
paint_color_encoded          0.012867
Name: rental_price_per_day, dtype: float64

In [39]:
import plotly.express as px
px.scatter(pricing_df, y='rental_price_per_day', color='rental_price_per_day')

In [42]:
pricing_df["rental_price_per_day"].mean()

121.21453644435267

# questions

- Which share of our owner’s revenue would potentially be affected by the feature?
- How many rentals would be affected by the feature depending on the threshold and scope we choose?

In [8]:
#threshold: how long should the minimum delay be?
#scope: should we enable the feature for all cars?, only Connect cars?
threshold = 60
scope_getaround_connect = True

In [18]:
delay_df[delay_df['time_delta_with_previous_rental_in_minutes'] > 0]

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
6,511639,370585,connect,ended,-15.0,563782.0,570.0
19,519491,312389,mobile,ended,58.0,545639.0,420.0
34,525044,349751,mobile,ended,NaN,510607.0,60.0
40,528808,181625,connect,ended,-76.0,557404.0,330.0
64,533670,320824,connect,ended,-6.0,556563.0,630.0
...,...,...,...,...,...,...,...
21269,568049,381499,connect,canceled,NaN,562174.0,720.0
21272,568241,396409,mobile,canceled,NaN,566136.0,570.0
21275,568523,297973,mobile,ended,12.0,567121.0,240.0
21286,569717,377312,mobile,ended,230.0,545045.0,90.0


In [19]:
# Filter out cancellations as they do not have a delay
rental_data_filtered = delay_df[delay_df['state'] == 'ended']
# Calculate the percentage of rentals that were late
# Calculate the average delay for rentals that were late
average_delay = rental_data_filtered[rental_data_filtered['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean()
average_delay

201.79147171416417

In [20]:
# Filter out rentals with delay at checkout in minutes greater than or equal to threshold
rentals_with_delay_threshold = rental_data_filtered[rental_data_filtered['time_delta_with_previous_rental_in_minutes'] >= threshold]

# Get unique car_id from rentals_with_delay_threshold
affected_cars = rentals_with_delay_threshold['car_id'].unique()

# Calculate the percentage of cars that would be affected
affected_cars_percentage = len(affected_cars) / rental_data_filtered['car_id'].nunique() * 100

# Print the affected cars and the percentage
print(f"Cars affected by a threshold set to {threshold}: {affected_cars}")
print(f"Percentage of cars affected by a threshold set to {threshold}: {affected_cars_percentage}%")

Cars affected by a threshold set to 60: [370585 312389 349751 181625 320824 404169 397814 352436 398802 409676
 358772 225795 369230 383309 411521 382901 380886 359880 385783 402993
 375915 245154 375126 289720 281955 310444 192179 241021 381123 195888
 382884 392147 380866 354515 390467 415953 314605 402528 365926 353425
 390927 358978 399528 227100 371387 401475 344091 408821 388767 384479
 175289 321582 265664 360474 374809 325695 385642 410393 356106 366071
 378931 333407 392813 374169 406999 214635 409750 383110 402677 385629
 229646 400945 212965 344254 392479 390509 310518 377781 407334 383329
 308579 337217 402235 277669 396552 409344 396250 298757 405611 333910
 358082 409589 387939 355788 311265 372779 334190 338940 389958 374684
 404192 389521 402209 382734 341105 339607 391519 380708 392368 295415
 397046 380647 361564 396009 381754 377509 385721 375553 386786 380019
 290109 405923 285241 406995 319886 412021 364783 374486 395598 310553
 385073 307439 346519 375834 393908 4